In [74]:
import numpy as np
import xarray as xr
import pandas as pd
from pathlib import Path

from bids import BIDSLayout

import nilearn
from nilearn import datasets, plotting, regions, maskers, image, connectome


In [75]:
ATLAS_NAME = 'dosenbach2010'

atlas = datasets.fetch_coords_dosenbach_2010(ordered_regions=True, legacy_format=False)

In [76]:
# load rs-fMRI session from the BIDS dataset

layout = BIDSLayout('data/julia2018_x5alias',
                    derivatives=True,
                    database_path='tmp/pybids_cache/julia2018')

# all_subjects = layout.get_subject()
TR = layout.get_tr(task='rest')

images = layout.get(task='rest',
                    desc='preproc',
                    suffix='bold',
                    scope='fmriprep',
                    extension='nii.gz',
                    return_type='filename')

mask_images = layout.get(task='rest',
                         desc='brain',
                         suffix='mask',
                         scope='fmriprep',
                         extension='nii.gz',
                         return_type='filename')

confounds_files = layout.get(task='rest',
                             desc='confounds',
                             suffix='timeseries',
                             scope='fmriprep',
                             extension='tsv',
                             return_type='filename')

confounds_cols = ['trans_x', 'trans_y', 'trans_z',
                  'csf', #'whitematter',
                  'rot_x', 'rot_y', 'rot_z',
                  'global_signal',
                  'a_comp_cor_00', 'a_comp_cor_01',
                  ]

In [78]:

# extract resting timeseries
import re
from tqdm import tqdm

timeseries = {}
subjects = []

for img, mask_img, confounds in tqdm(zip(images, mask_images, confounds_files)):

    subject = re.search('func/sub-(.*)_ses', img)[1]
    subjects.append(subject)
    
    output_path = Path('tmp') / f'{ATLAS_NAME}_cache/' / f'{subject}_timeseries.npz'

    if output_path.exists():
        # print(f'Loading {subject} from cache...')
        ts = np.load(output_path)['arr_0']
        timeseries[subject] = ts
        continue

    masker = maskers.NiftiSpheresMasker(
        seeds=atlas['rois'].values,
        smoothing_fwhm=6,
        radius=5,
        allow_overlap=False,
        detrend=True,
        standardize=True,
        low_pass=0.08,
        high_pass=0.009,
        t_r=TR,
        verbose=0)

    confounds = pd.read_csv(confounds, sep='\t')[confounds_cols].values
    ts = masker.fit_transform(img, confounds=confounds)
    
    np.savez(output_path, ts)
    timeseries[subject] = ts

# reshape dim-0 is subject, dim-1 is region, dim-2 is time point
timeseries = np.array(list(timeseries.values())).transpose(0,2,1)


32it [00:00, 807.35it/s]


In [80]:

atlas_regions = pd.concat(
  [pd.DataFrame(v) for k, v in atlas.items() if k != 'description'],
  axis=1)

atlas_regions.rename(columns={'x': 'mni152_x',
                              'y': 'mni152_y',
                              'z': 'mni152_z',
                              0: 'label',
                              }, inplace=True)
atlas_regions.set_index('label', inplace=True)
atlas_regions.index.name = 'region'

ds = xr.Dataset({
  'timeseries': (['subject', 'region', 'timestep'], timeseries)
})

# metadata
ds.coords['timestep']  = range(0,125)
ds.attrs['description'] = 'Parcellated resting-state using Dosenbach2010 coordinates'
ds.coords['subject'].attrs['description'] = ('subject identifier; AVGPxx for action video'
                                             'gamers and NVGPxx for non-video gamers')

# subject data
bids_participants = pd.read_csv('data/julia2018/participants.tsv', sep='\t')
bids_participants.rename(columns={'participant_id': 'subject'}, inplace=True)

bids_participants['subject'] = bids_participants['subject'].apply(
  lambda x: x.split('-')[1])

bids_participants.set_index('subject', inplace=True)
bids_participants = bids_participants.query('index in @subjects')

# # merge all datasets and store
ds = xr.merge([ds, bids_participants.to_xarray(), atlas_regions.to_xarray()])
ds.to_netcdf(f'data/julia2018_resting/timeseries_{ATLAS_NAME}.nc', engine='netcdf4')
ds

<xarray.Dataset>
Dimensions:                      (subject: 32, region: 160, timestep: 125)
Coordinates:
  * subject                      (subject) object 'AVGP01' ... 'NVGP19NEW'
  * region                       (region) object 'temporal 82' ... 'post cing...
  * timestep                     (timestep) int64 0 1 2 3 4 ... 121 122 123 124
Data variables: (12/13)
    timeseries                   (subject, region, timestep) float64 0.06018 ...
    age                          (subject) float64 27.0 19.0 23.0 ... 19.0 22.0
    sex                          (subject) object 'M' 'M' 'M' ... 'M' 'M' 'M'
    group                        (subject) object 'AVGP' 'AVGP' ... 'NVGP'
    preprocessed_beh             (subject) bool True True True ... True True
    preprocessed_rsfmri          (subject) bool True True True ... True True
    ...                           ...
    inverse_efficiency_score_ms  (subject) float64 1.355e+03 981.0 ... 1.379e+03
    false_alarm_pct              (subject) float64 3.13 0.0 0.0 ... 70.31 3.13
    mni152_x                     (region) int64 18 -21 -6 33 ... -54 59 43 -55
    mni152_y                     (region) int64 -81 -79 -79 -73 ... -22 -13 1 7
    mni152_z                     (region) int64 -33 -33 -33 -30 ... 9 8 12 23
    network                      (region) object 'cerebellum' ... 'sensorimotor'
Attributes:
    description:  Parcellated resting-state using Dosenbach2010 coordinates